In [196]:
from pywr.core import Model
from pywr.notebook import draw_graph
import matplotlib.pyplot as plt
import os
from pywr.domains.groundwater import KeatingAquifer
from pywr.parameters.groundwater import KeatingStreamFlowParameter
from pywr.core import Model, Input, Output, Link, Storage, Node, Catchment
from pywr.recorders import (
    NumpyArrayNodeRecorder,
    NumpyArrayLevelRecorder,
    NumpyArrayStorageRecorder,
)
import pandas
import numpy as np
import pytest

def createAquifer(model,name):
    num_streams = 1
    num_additional_inputs = 0
    stream_flow_levels = [[1.0, 10.0]]  # m
    transmissivity = [100, 2000]  # m2/d
    transmissivity = [t * 0.0001 for t in transmissivity]  # m3 to Ml
    coefficient = 10  # no units
    storativity = [0.0005]  # % este parámetro es clave
    levels = [1.0, 10.0]  # m
    area = 5000 * 5000  # m2
    
    aqfer=KeatingAquifer(
        model,
        name,
        num_streams,
        num_additional_inputs,
        stream_flow_levels,
        transmissivity,
        coefficient,
        levels,
        area=area,
        storativity=storativity)

    aqfer.initial_level=2
    
    return aqfer

def connectAquifer(modelo,acuifero,nodosRecarga,nodosDescarga):
    for nodoR in nodosRecarga:
        model.nodes[nodoR].connect(acuifero)
    
    for nodoD in nodosDescarga:
        acuifero.connect(modelo.nodes[nodoD],from_slot=0)
    
    return modelo,acuifero
    
def connectNodes(modelo,entradas,salidas):
    
    for ind,entrada in enumerate(entradas):
        modelo.nodes[entrada].connect(model.nodes[salidas[ind]])
           
    return modelo

def getDischarge(lista):
    return ['deficit'+x.replace('REM','').replace('Sub','') for x in lista]

def getOutputs(lista):
    return [x.replace('deficit','')+'Mix' for x in lista]

if __name__ == "__main__":
    
    path="testREM.json"
    model = Model.load(path)
    
#     crear objetos acuíferos
    aqferChan=createAquifer(model,'AC02_CHA')
    aqferNuble=createAquifer(model,'AC03_NUB')

#     conectar nodos existentes al acuifero Changaral
    recargas=['REMGYMSub']
    
    descargas=['abstraccionAqCha','GreeneYMairaMix']
    
    model,aqferChan=connectAquifer(model,aqferChan,recargas,descargas)
    
#     conectar nodos existentes al acuifero Ñuble
    recargas=['REMCollicoSub']
    
    descargas=['abstraccionAqNub']+getDischarge(recargas)
    
    model,aqferNiquen=connectAquifer(model,aqferNuble,
recargas,descargas)
        
#     conectar los nodos de déficit que faltan
    entradas=['deficitCollico']
    
    salidas=getOutputs(entradas)

#     conectar los sitios de bombeo de aguas subterráneas
    model=connectNodes(model,entradas,salidas)

    plt.figure()
    draw_graph(model,labels=True,attributes=False,
               width=700,height=700)
    plt.show()

<IPython.core.display.Javascript object>

<Figure size 640x480 with 0 Axes>

In [197]:
# correr modelo
model.solver.name="glpk-edge"
# run the model and see if it works
run_stats = model.run()
print(run_stats)

Model executed 1 scenarios in 0.1 seconds, running at 543.3 timesteps per second.


In [198]:
print(model.nodes["NubleSanFabian"].flow)
print(model.nodes["bocatoma2"].flow)
print(model.nodes["FRCollico"].flow)
print(model.nodes["CollicoMix"].flow)
print(model.nodes["CollicoSup"].flow)
print(model.nodes["REMCollico"].flow)
print(model.nodes["deficitCollico"].flow)

[51382.08]
[51382.08]
[51382.08]
[0.0001323]
[51382.08]
[0.]
[0.]
